ドキュメントが大きすぎるとモデルにfeedしきれないことある。
そのためsplit the doc into chunks
- 特定のキャラクターでsplit
- トークン数でsplit

CharacterTextSplitter()のchunk_sizeはlangchainのマニュアルでの説明を見つけることができなかった。以下の見ると、
まず、文字をchunk_sizeまで取り出して、そこから一番近いセパレータを探して、そこまでを1つのchunkにするようである。

https://stackoverflow.com/questions/76633836/what-does-langchain-charactertextsplitters-chunk-size-param-even-do

Split by Character
- CharacterTextSplitter(separator="\n\n",chunk_size=1000)でスプリッター作成
- スプリッター.create_documentでスプリット。一つ一つは<class 'langchain_core.documents.base.Document'>

Split by Token
- CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500)でスプリッター作成
- スプリッター.split_text()でスプリット。一つ一つはstr

1つ目の方法だと、少なくとも3つ目までのchunkは同じ行が含まれていない。2つ目の方法だと、1つ目のchunkの後半の部分が2つ目のchunkに含まれている。

In [2]:
path = '..//langchain-course-main/June 2024 Update/01-Data-Connections/'

with open(path+'some_data/FDR_State_of_Union_1944.txt') as file:
    speech_text = file.read()

#  Document Transformers

In [3]:
# Characters
len(speech_text)

21927

In [4]:
# Words
len(speech_text.split())

3750

## Split by Character

In [5]:
from langchain.text_splitter import CharacterTextSplitter

In [36]:
text_splitter = CharacterTextSplitter(separator="\n\n",chunk_size=1000) #1000 is default value

In [37]:
texts = text_splitter.create_documents([speech_text])
print(type(texts))
print('\n')
print(type(texts[0]))
print('\n')
print(texts[0])

<class 'list'>


<class 'langchain_core.documents.base.Document'>


page_content='This Nation in the past two years has become an active partner in the world's greatest war against human slavery.

We have joined with like-minded people in order to defend ourselves in a world that has been gravely threatened with gangster rule.

But I do not think that any of us Americans can be content with mere survival. Sacrifices that we and our allies are making impose upon us all a sacred obligation to see to it that out of this war we and our children will gain something better than mere survival.

We are united in determination that this war shall not be followed by another interim which leads to new disaster- that we shall not repeat the tragic errors of ostrich isolationism—that we shall not repeat the excesses of the wild twenties when this Nation went for a joy ride on a roller coaster which ended in a tragic crash.'


In [8]:
len(texts[0].page_content)

841

In [9]:
texts[1]

Document(page_content='When Mr. Hull went to Moscow in October, and when I went to Cairo and Teheran in November, we knew that we were in agreement with our allies in our common determination to fight and win this war. But there were many vital questions concerning the future peace, and they were discussed in an atmosphere of complete candor and harmony.\n\nIn the last war such discussions, such meetings, did not even begin until the shooting had stopped and the delegates began to assemble at the peace table. There had been no previous opportunities for man-to-man discussions which lead to meetings of minds. The result was a peace which was not a peace. That was a mistake which we are not repeating in this war.\n\nAnd right here I want to address a word or two to some suspicious souls who are fearful that Mr. Hull or I have made "commitments" for the future which might pledge this Nation to secret treaties, or to enacting the role of Santa Claus.')

In [10]:
len(texts[1].page_content)

933

In [11]:
type(texts[0])

langchain_core.documents.base.Document

In [30]:
print(len(texts),'\n')
for i in texts:
  print(len(i.page_content), " ", len(i.page_content.split()))

28 

841   152
933   169
900   145
904   144
905   150
672   112
950   159
977   172
497   84
944   156
969   167
877   150
660   113
855   149
829   144
866   147
878   149
727   139
874   154
588   93
504   80
609   104
886   144
882   149
943   165
972   163
782   137
892   166


In [45]:
print(texts[0].page_content)

This Nation in the past two years has become an active partner in the world's greatest war against human slavery.

We have joined with like-minded people in order to defend ourselves in a world that has been gravely threatened with gangster rule.

But I do not think that any of us Americans can be content with mere survival. Sacrifices that we and our allies are making impose upon us all a sacred obligation to see to it that out of this war we and our children will gain something better than mere survival.

We are united in determination that this war shall not be followed by another interim which leads to new disaster- that we shall not repeat the tragic errors of ostrich isolationism—that we shall not repeat the excesses of the wild twenties when this Nation went for a joy ride on a roller coaster which ended in a tragic crash.


In [46]:
print(texts[1].page_content)

When Mr. Hull went to Moscow in October, and when I went to Cairo and Teheran in November, we knew that we were in agreement with our allies in our common determination to fight and win this war. But there were many vital questions concerning the future peace, and they were discussed in an atmosphere of complete candor and harmony.

In the last war such discussions, such meetings, did not even begin until the shooting had stopped and the delegates began to assemble at the peace table. There had been no previous opportunities for man-to-man discussions which lead to meetings of minds. The result was a peace which was not a peace. That was a mistake which we are not repeating in this war.

And right here I want to address a word or two to some suspicious souls who are fearful that Mr. Hull or I have made "commitments" for the future which might pledge this Nation to secret treaties, or to enacting the role of Santa Claus.


In [12]:
# text_splitter = CharacterTextSplitter(separator="\n\n") #chunk_size is minimum length, notice at 2000 still contains a \n\n

In [47]:
print(texts[2].page_content)

To such suspicious souls—using a polite terminology—I wish to say that Mr. Churchill, and Marshal Stalin, and Generalissimo Chiang Kai-shek are all thoroughly conversant with the provisions of our Constitution. And so is Mr. Hull. And so am I.

Of course we made some commitments. We most certainly committed ourselves to very large and very specific military plans which require the use of all Allied forces to bring about the defeat of our enemies at the earliest possible time.

But there were no secret treaties or political or financial commitments.

The one supreme objective for the future, which we discussed for each Nation individually, and for all the United Nations, can be summed up in one word: Security.

And that means not only physical security which provides safety from attacks by aggressors. It means also economic security, social security, moral security—in a family of Nations.


# Split by Token

Most LLMs operate on a token limit (although some price based on characters!). Let's explore how to split based on tokens.

In [ ]:
# !pip install tiktoken

In [38]:
text_splitter2 = CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500) #now chunk size is a hard length based on tokens

In [39]:
texts2 = text_splitter2.split_text(speech_text)

In [41]:
print(type(texts2))
print(type(texts2[0]))

<class 'list'>
<class 'str'>


In [34]:
texts2[0]

'This Nation in the past two years has become an active partner in the world\'s greatest war against human slavery.\n\nWe have joined with like-minded people in order to defend ourselves in a world that has been gravely threatened with gangster rule.\n\nBut I do not think that any of us Americans can be content with mere survival. Sacrifices that we and our allies are making impose upon us all a sacred obligation to see to it that out of this war we and our children will gain something better than mere survival.\n\nWe are united in determination that this war shall not be followed by another interim which leads to new disaster- that we shall not repeat the tragic errors of ostrich isolationism—that we shall not repeat the excesses of the wild twenties when this Nation went for a joy ride on a roller coaster which ended in a tragic crash.\n\nWhen Mr. Hull went to Moscow in October, and when I went to Cairo and Teheran in November, we knew that we were in agreement with our allies in our

In [35]:
print(len(texts2),'\n')
for i in texts2:
  print(len(i), " ", len(i.split()))

15 

2332   411
2424   401
2166   359
2199   370
2096   359
2036   342
1850   313
2131   366
2354   402
2087   370
2193   386
2381   392
2332   395
2213   379
1676   303


In [42]:
print(texts2[0],'\n')

This Nation in the past two years has become an active partner in the world's greatest war against human slavery.

We have joined with like-minded people in order to defend ourselves in a world that has been gravely threatened with gangster rule.

But I do not think that any of us Americans can be content with mere survival. Sacrifices that we and our allies are making impose upon us all a sacred obligation to see to it that out of this war we and our children will gain something better than mere survival.

We are united in determination that this war shall not be followed by another interim which leads to new disaster- that we shall not repeat the tragic errors of ostrich isolationism—that we shall not repeat the excesses of the wild twenties when this Nation went for a joy ride on a roller coaster which ended in a tragic crash.

When Mr. Hull went to Moscow in October, and when I went to Cairo and Teheran in November, we knew that we were in agreement with our allies in our common de

In [43]:
print(texts2[1],'\n')

And right here I want to address a word or two to some suspicious souls who are fearful that Mr. Hull or I have made "commitments" for the future which might pledge this Nation to secret treaties, or to enacting the role of Santa Claus.

To such suspicious souls—using a polite terminology—I wish to say that Mr. Churchill, and Marshal Stalin, and Generalissimo Chiang Kai-shek are all thoroughly conversant with the provisions of our Constitution. And so is Mr. Hull. And so am I.

Of course we made some commitments. We most certainly committed ourselves to very large and very specific military plans which require the use of all Allied forces to bring about the defeat of our enemies at the earliest possible time.

But there were no secret treaties or political or financial commitments.

The one supreme objective for the future, which we discussed for each Nation individually, and for all the United Nations, can be summed up in one word: Security.

And that means not only physical security 

In [48]:
print(texts2[2],'\n')

All our allies have learned by bitter experience that real development will not be possible if they are to be diverted from their purpose by repeated wars—or even threats of war.

China and Russia are truly united with Britain and America in recognition of this essential fact:

The best interests of each Nation, large and small, demand that all freedom-loving Nations shall join together in a just and durable system of peace. In the present world situation, evidenced by the actions of Germany, Italy, and Japan, unquestioned military control over disturbers of the peace is as necessary among Nations as it is among citizens in a community. And an equally basic essential to peace is a decent standard of living for all individual men and women and children in all Nations. Freedom from fear is eternally linked with freedom from want.

There are people who burrow through our Nation like unseeing moles, and attempt to spread the suspicion that if other Nations are encouraged to raise their sta